# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-21 02:30:21] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-21 02:30:21] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-21 02:30:21] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-21 02:30:24] INFO server_args.py:1670: Attention backend not specified. Use fa3 backend by default.


[2026-01-21 02:30:24] INFO server_args.py:2570: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=54.48 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=80 avail_mem=54.33 GB):  20%|██        | 4/20 [00:00<00:01,  8.94it/s]

Capturing batches (bs=32 avail_mem=54.27 GB):  50%|█████     | 10/20 [00:00<00:00, 18.18it/s]

Capturing batches (bs=4 avail_mem=54.25 GB):  80%|████████  | 16/20 [00:01<00:00, 21.19it/s]

Capturing batches (bs=1 avail_mem=54.24 GB): 100%|██████████| 20/20 [00:01<00:00, 17.89it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Steven. I'm a science fiction writer, and I'm using the following prompts for the story I'm working on. Write a first draft of the story in this format. My name is Steven, and I am a science fiction writer. Here are the prompts I have been given to write a story, which will be 8000 words long:

1. I have a character named Sarah who is studying robotics in college and has a set of tools that she uses for her experiments. One of the tools is a large robot that she uses to make copies of herself. I have written a rough draft of her character with her name
Prompt: The president of the United States is
Generated text:  from the______. A) House of Representatives B) Senate C) Executive branch D) Legislature.
Answer:

C

Which of the following statements about the process of photosynthesis is incorrect?
A. During photosynthesis, chlorophyll absorbs light energy, producing ATP and NADPH.
B. In the dark reaction, only chlorophyll participates in the li

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I am a [job title] at [company name]. I am passionate about [reason for being at the company]. I am always looking for ways to [what I enjoy doing at the company]. I am a [any other relevant qualities or skills that make me unique]. I am excited to [any future goals or aspirations you have]. Thank you for having me! [Name] [Company name] is a [brief description of the company]. I am excited to meet you and learn more about your company. [Name] [Company name] is a [brief description of the company]. I am excited to meet

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as the City of Light. It is the largest city in France and the second-largest city in the European Union. Paris is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and Louvre Museum, as well as its rich history and cultural heritage. The city is also known for its fashion industry, art scene, and its role in the French Revolution and French Revolution-era architecture. Paris is a popular tourist destination and a major economic center in France. It is home to many world-renowned museums, theaters, and restaurants. The city is also home to the French Parliament and the

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction and impact on society. Here are some of the most likely trends that are expected to shape the future of AI:

1. Increased integration with human intelligence: One of the most significant trends in AI is the increasing integration of AI with human intelligence. This could lead to more sophisticated and personalized AI systems that can better understand and respond to human emotions and behaviors.

2. Enhanced privacy and security: As AI systems become more sophisticated, there is a risk that they may be used for malicious purposes. Therefore, it is important to ensure that AI systems are designed



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Your Name], and I'm a [Your Profession] who has always had a love for [Your Passion]. I'm passionate about [Your Passion], and I've always wanted to share my knowledge and experience with others. Whether it's through speaking, teaching, or just being an inspiration, I'm here to help anyone who asks. Let me know if you're interested in learning more about me. 🌟✨✨✨

Are you interested in learning more about me? Write your query below so I can provide you with a personalized response. Remember, your query will help me tailor my response to your specific interests. �

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. 

To elaborate, Paris is the largest city in the European Union and the second-most populous city in the world, with over 2 million residents. It is known for its classical architecture, vibrant

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

],

 and

...

 AI

 is

 used

 in

 a

In [6]:
llm.shutdown()